## Quick Return Mechanism

In [ ]:
from sympy import *
import sympy.physics.mechanics as me
me.init_vprinting()

### EoM generation

In [ ]:
q1, q2, q3, q4, q5 = me.dynamicsymbols("q1, q2, q3, q4, q5")
q1d, q2d, q3d, q4d, q5d = me.dynamicsymbols("q1, q2, q3, q4, q5", 1)
u1, u2, u3, u4, u5 = me.dynamicsymbols("u1, u2, u3, u4, u5")
d1, d2, r1, r2, L = symbols("d1, d2, r1, r2, L")

N, A, B, C = symbols("N, A, B, C", cls=me.ReferenceFrame)
A.orient_axis(N, N.z, q1)
B.orient_axis(N, N.z, q2)
C.orient_axis(N, N.z, q3)

O, P1, P2, Q, R1, R2, M = symbols("O, P1, P2, Q, R1, R2, M", cls=me.Point)
M.set_pos(O, -d1 * N.y)
P1.set_pos(O, r1 * A.x)
P2.set_pos(M, q4 * B.x)
Q.set_pos(M, L * B.x)
R1.set_pos(Q, r2 * C.x)
R2.set_pos(O, q5 * N.x + d2 * N.y)

G1 = O.locatenew("G1", r1 / 2 * A.x)
G2 = M.locatenew("G2", L / 2 * B.x)
G3 = Q.locatenew("G3", r2 / 2 * C.x)

O.set_vel(N, 0)
M.set_vel(N, 0)
P1.v2pt_theory(O, N, A)
P2.set_vel(N, u4 * B.x)
Q.v2pt_theory(M, N, B)
R1.v2pt_theory(Q, N, C)
R2.set_vel(N, u5 * N.x)
G1.v2pt_theory(O, N, A)
G2.v2pt_theory(M, N, B)
G3.v2pt_theory(Q, N, C)

In [ ]:
# constraints
zero1 = P1.pos_from(O) + O.pos_from(P2)
zero2 = R1.pos_from(O) + O.pos_from(R2)
f_c = [zero1 & N.x, zero1 & N.y, zero2 & N.x, zero2 & N.y]

t = symbols("t")
dzero1 = zero1.diff(t, N)
dzero2 = zero2.diff(t, N)
f_v = [dzero1 & N.x, dzero1 & N.y, dzero2 & N.x, dzero2 & N.y]

q_ind = [q1]
u_ind = [u1]
q_dep = [q2, q3, q4, q5]
u_dep = [u2, u3, u4, u5]

kde = [q1d - u1, q2d - u2, q3d - u3, q4d - u4, q5d - u5]

kane = me.KanesMethod(
    N,
    q_ind=q_ind,
    u_ind=u_ind,
    q_dependent=q_dep,
    u_dependent=u_dep,
    kd_eqs=kde,
    configuration_constraints=f_c,
    velocity_constraints=f_v
)

In [ ]:
m, ms = symbols("m, m_s")
inertia_dyac_crank = me.inertia(A, 0, 0, m * r1**2 / 12)
inertia_dyac_rod_MQ = me.inertia(A, 0, 0, m * L**2 / 12)
inertia_dyac_rod_QR1 = me.inertia(A, 0, 0, m * r2**2 / 12)

crank = me.RigidBody("Crank", G1, A, m, (inertia_dyac_crank, G1))
rod_MQ = me.RigidBody("Rod MQ", G2, B, m, (inertia_dyac_rod_MQ, G2))
rod_QR1 = me.RigidBody("Rod QR1", G3, C, m, (inertia_dyac_rod_QR1, G3))
slider1 = me.Particle("Slider 1", P2, ms)
slider2 = me.Particle("Slider 2", R2, ms)

bodies = [crank, rod_MQ, rod_QR1, slider1, slider2]

g = symbols("g")
loads = [
    (G1, -m * g * N.y),
    (G2, -m * g * N.y),
    (G3, -m * g * N.y),
    (P2, -ms * g * N.y),
    (R2, -ms * g * N.y),
]

fr, frstar = kane.kanes_equations(bodies, loads)

In [ ]:
trigsimp(fr + frstar)

### Find Initial Conditions for the configuration

In [ ]:
import numpy as np
from spb import plot_contour

constants = [r1, r2, L, m, ms, g, d1, d2]
values = [5, 5, 15, 12, 12, 9.81, 8, 8]
constants = dict(zip(constants, values))

# substitution dictionary
sd = {
    q1: np.deg2rad(165)
}
sd.update(constants)

# visualize proper initial guesses for solving q2, q4
q2q4_eq = [c.subs(sd) for c in f_c[:2]]
plot_contour(*q2q4_eq, (q2, 0, 2*pi), (q4, 0, sd[L]), {"levels": [0]}, is_filled=False)

In [ ]:
nsolve(q2q4_eq, [q2, q4], [1.8, 12])

In [ ]:
# update the substitution dictionary with the newly found values
sd.update({q2: 2.05003605544231, q4: 10.47404046241})

# visualize proper initial guesses for solving q3, q5
q3q5_eq = [c.subs(sd) for c in f_c[2:]]
plot_contour(*q3q5_eq, (q3, 0, 2*pi), (q5, -10, 10), {"levels": [0]}, is_filled=False)

In [ ]:
nsolve(q3q5_eq, [q3, q5], [0.5, -2])

In [ ]:
sd.update({q3: 0.568018505594562, q5: -2.70172728198401})

### Simulation

In [ ]:
import numpy as np
from pydy.system import System

# remove unused symbols, otherwise Pydy raises error
constants.pop(d1, None)
constants.pop(d2, None)

sys = System(kane)
sys.constants = constants
sys.initial_conditions = {
    q1: sd[q1],
    q2: sd[q2],
    q3: sd[q3],
    q4: sd[q4],
    q5: sd[q5],
    u1: 0, u2: 0, u3: 0, u4: 0, u5: 0
}

fps = 60
t0, tf = 0, 15
n = int(fps * (tf - t0))
sys.times = np.linspace(t0, tf, n)

results = sys.integrate()

In [ ]:
import matplotlib.pyplot as plt

coordinates = [q1, q2, q3, q4, q5]
speeds = [u1, u2, u3, u4, u5]

fig, ax = plt.subplots(2, 1, sharex=True)
ax[0].plot(sys.times, results[:, :5])
ax[0].set_ylabel('Coordinates')
ax[0].legend(["${}$".format(latex(c)) for c in coordinates])
ax[1].plot(sys.times, results[:, 5], label="$u_{1}(t)$ [rad / s]")
ax[1].plot(sys.times, results[:, 9], label="$u_{5}(t)$ [m / s]")
ax[1].legend()
ax[1].set_xlabel('Time [s]')
ax[1].set_ylabel('Velocities')

In [ ]:
from pydy.viz import Scene, VisualizationFrame, Cylinder, Cube

radius = 0.2
crank_geom = Cylinder(r1, radius, name="crank geom", color="gray")
rod_MQ_geom = Cylinder(L, radius, name="rod MQ geom", color="gray")
rod_QR1_geom = Cylinder(r2, radius, name="rod QR1 geom", color="gray")
slider_q4 = Cube(3 * radius, name="Slider q4", color="red")
slider_q5 = Cube(3 * radius, name="Slider q5", color="red")

rf1, rf2, rf3 = symbols("rf1, rf2, rf3", cls=me.ReferenceFrame)
rf1.orient_axis(A, A.z, -pi / 2)
rf2.orient_axis(B, B.z, -pi / 2)
rf3.orient_axis(C, C.z, -pi / 2)

crank_vf = VisualizationFrame("Crank VF", rf1, G1, crank_geom)
rod_MQ_vf = VisualizationFrame("Rod MQ VF", rf2, G2, rod_MQ_geom)
rod_QR1_vf = VisualizationFrame("Rod QR1 VF", rf3, G3, rod_QR1_geom)
slider_q4 = VisualizationFrame("Slider q4", rf2, P2, slider_q4)
slider_q5 = VisualizationFrame("Slider q5", N, R2, slider_q5)

# reinsert the previously removed values in order to visualize the system
sys.constants.update({d1: 8, d2: 8})

scene = Scene(N, O, crank_vf, rod_MQ_vf, rod_QR1_vf, slider_q4, slider_q5, times=sys.times)
scene.states_symbols = coordinates + speeds
scene.constants = sys.constants
scene.states_trajectories = results
scene.display_jupyter(axes_arrow_length=5)